LIBRERIAS

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time
from datetime import datetime, timedelta
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options

Funciones

In [3]:
def es_entero(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
def es_partido_dobles(linea):
    posibles_nombres = linea[2:7]

    nombres_detectados = [
        n for n in posibles_nombres
        if '.' in n and len(n.split()) <= 2 and 'atp' not in n.lower() and 'wta' not in n.lower()
    ]

    return len(nombres_detectados) >= 4

ÚLTIMA VERSIÓN DEL CÓDIGO

In [5]:
try:
    driver.quit()
except:
    pass  # Por si driver aún no estaba creado

options = Options()
#options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
driver.get("https://www.sofascore.com/tennis")

try:
    wait = WebDriverWait(driver, 10)

    # Eliminar overlays
    def wait_overlay_to_disappear(driver, wait, class_name="fc-dialog-overlay", timeout=10):
        try:
            wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, class_name)))
        except TimeoutException:
            driver.execute_script("""
                let overlays = document.getElementsByClassName(arguments[0]);
                while(overlays.length > 0) {
                    overlays[0].parentNode.removeChild(overlays[0]);
                }
            """, class_name)
            time.sleep(1)

    #Esto que está comentado es para eliminar pestañas emergentes de manera automática (no sirve para captcha)
    """
    # Aceptar cookies u otros popups
    buttons = driver.find_elements(By.TAG_NAME, "button")
    print(len(buttons), buttons)
    for i in range(len(buttons)):
        try:
            buttons = driver.find_elements(By.TAG_NAME, "button")
            btn = buttons[i]
            text = btn.text.lower()
            if any(word in text for word in ["aceptar", "accept", "consent", "confirm", "consentir", "si"]):
                wait_overlay_to_disappear(driver, wait)
                wait.until(EC.element_to_be_clickable(btn))
                driver.execute_script("arguments[0].scrollIntoView();", btn)
                time.sleep(10)
                btn.click()
                break
        except StaleElementReferenceException:
            continue
        except Exception:
            continue"""

    # Esperar manualmente a que se resuelva el CAPTCHA
    input("Resuelve el CAPTCHA (y otras cosas) manualmente en el navegador y presiona Enter aquí para continuar...")

    time.sleep(2)

    # Variables
    base_url = "https://www.sofascore.com/es/tenis"
    fecha_actual = datetime(2025, 4, 8)
    datos = []

    for i in range(10):
        fecha = fecha_actual - timedelta(days=i)
        fecha_str = fecha.strftime('%Y-%m-%d')
        url = f"{base_url}/{fecha_str}"
        print("Navegando a URL:", url)
        driver.get(url)
        print("dentro")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.Box.klGMtt")))
        pista = driver.find_elements(By.CSS_SELECTOR, "div.Box.klGMtt")

        enlaces_partidos = []
        torneo_actual = {}

        #print("Número de pistas", len(pista))
        for p in pista:
            texto = p.text.lower()
            lineas = p.text.split("\n")

            # Detectar y almacenar torneo actual
            if not any(x in texto for x in ["matches"]) and any(x in texto for x in ["grand slam", "atp", "wta", "open", 'challenger', 'itf', 'utr']):
                if len(lineas) == 3:
                    tipo = lineas[0]
                    if tipo == 'Grand Slam':
                        lugar_partes = [s.strip() for s in lineas[1].split(",")]
                        nombre = lugar_partes[0]
                        ciudad = lugar_partes[1] if len(lugar_partes) > 1 else ""
                        pais = lugar_partes[2] if len(lugar_partes) > 2 else ""
                        especificacion = lugar_partes[3] if len(lugar_partes) > 3 else "Singles"
                        superficie = lineas[2]
                        if ciudad in ['Boys', 'Girls'] or pais == 'Doubles':
                            ciudad = 'Paris'
                            pais = 'France'
                            especificacion = 'Doubles'
                    else:
                        tipo = lineas[0]
                        lugar_partes = [s.strip() for s in lineas[1].split(",")]
                        ciudad = lugar_partes[0]
                        pais = lugar_partes[1] if len(lugar_partes) > 1 else ""
                        especificacion = lugar_partes[2] if len(lugar_partes) > 2 else "Singles"
                        superficie = lineas[2]
                        nombre = ciudad

                    torneo_actual = {
                        "tipo": tipo,
                        "nombre": nombre,
                        "ciudad": ciudad,
                        "pais": pais,
                        "especificacion": especificacion,
                        "superficie": superficie
                    }

                    #print("Torneo detectado:", torneo_actual)

            # Extraer enlace si es un partido
            if not any(x in texto for x in ["grand slam", "atp", "wta", "open", 'challenger', 'itf', 'utr', 'matches']) and texto.strip() != "":
                links = p.find_elements(By.TAG_NAME, "a")
                if links:
                    href = links[0].get_attribute("href")
                    enlaces_partidos.append((href, torneo_actual.copy()))  # Guardamos una copia del torneo actual

        print("Número de enlaces", len(enlaces_partidos))
        
        for href, torneo_info in enlaces_partidos:
            try:
                driver.get(href)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.d_flex.flex-d_column.gap_sm.p_sm")))
                est = driver.find_elements(By.CSS_SELECTOR, "div.d_flex.flex-d_column.gap_sm.p_sm")

                cuotaA = cuotaB = fecha_texto = dia = hora = dur = genero = antes = regla = setA = setB = nSets = "-"
                cancelado = "no"
                h2h1 = h2h2 = 0
                duraciones = []
                jugadores = []
                ranking = []
                sets = []

                for e in est:
                    lineas = e.text.split("\n")
                    #print("Número de lineas", len(lineas))
                    #print(lineas)  # lineas equivale a un partido

                    for i, line in enumerate(lineas):
                        #print(i, line)

                        if 'odds' in line.lower() or 'cuotas' in line.lower():
                            if i >= 3:
                                cuotaA = lineas[i - 3]
                                cuotaB = lineas[i - 1]

                        if "ft" in line.lower() or "retired" in line.lower() or 'final' in line.lower():
                            duraciones.append(lineas[i].strip())

                        if es_partido_dobles(lineas):
                            if "." in line and i >= 2 and i <= 7 and not ("atp" in line.lower() or "wta" in line.lower()): 
                                jugadores.append(line)
                        else:
                            if 'wta' in line.lower():
                                if i >= 1:
                                    jugadores.append(lineas[i - 1])
                                ranking.append(lineas[i])
                                antes = i
                                if genero == '-':
                                    genero = 'femenino'

                            if 'atp' in line.lower():
                                if i >= 1:
                                    jugadores.append(lineas[i - 1])
                                ranking.append(lineas[i])
                                antes = int(i)
                                if genero == '-':
                                    genero = 'masculino'

                        if "ft" in line.lower() or "retired" in line.lower() or 'final' in line.lower():
                            es, dur = lineas[i].split(" - ")

                        if "cancelado" in line.lower() or "cancelled" in line.lower() or "canceled" in line.lower():
                            cancelado = "si"

                        setA = lineas[len(lineas) - 2]
                        setB = lineas[len(lineas) - 1]

                        if len(lineas) > 0 and "/" in lineas[0]:
                            dia = lineas[0]
                        if len(lineas) > 1 and ":" in lineas[1]:
                            hora = lineas[1]

                # Determinar ganador
                ganador = -1
                try:
                    if setA.isdigit() and setB.isdigit():
                        if int(setA) > int(setB):
                            ganador = 0
                        elif int(setB) > int(setA):
                            ganador = 1
                except:
                    pass


                # Guardar en lista de diccionarios
                datos.append({
                    "tipo_torneo": torneo_info.get("tipo", ""),
                    "nombre_torneo": torneo_info.get("nombre", ""),
                    "ciudad_torneo": torneo_info.get("ciudad", ""),
                    "pais_torneo": torneo_info.get("pais", ""),
                    "especificacion_torneo": torneo_info.get("especificacion", ""),
                    "superficie_torneo": torneo_info.get("superficie", ""),
                    "estado_partido": es if 'es' in locals() else "-",
                    "jugadorA": jugadores[0] if len(jugadores) > 0 else "-",
                    "jugadorB": jugadores[1] if len(jugadores) > 1 else "-",
                    "jugadorA2": jugadores[2] if len(jugadores) > 2 else "-",
                    "jugadorB2": jugadores[3] if len(jugadores) > 3 else "-",
                    "cuotaA": cuotaA,
                    "cuotaB": cuotaB,
                    "duracion": dur,
                    "duracion_sets": " | ".join(duraciones) if duraciones else "-",
                    "setsA": setA,
                    "setsB": setB,
                    "ranking_jugadorA": ranking[0] if len(ranking) > 0 else "-",
                    "ranking_jugadorB": ranking[1] if len(ranking) > 1 else "-",
                    "genero": genero,
                    "dia": dia,
                    "hora": hora,
                    "cancelado": cancelado,
                    "ganador": ganador
                })

            except Exception as e:
                print(f"Error al procesar {href}: {e}")

finally:
    driver.quit()
    # Exportar a CSV
    df = pd.DataFrame(datos)
    display(df)
    #df.to_csv("partidos_tenis2.csv", index=False)
    print("Datos guardados de", fecha_actual)

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=56304): Read timed out. (read timeout=120)

Nuevo dataset

In [ ]:
df.to_csv("data/partidos_tenis_04_08-04_XX.csv", index=False)

In [4]:
datos = pd.read_csv("data/partidos_tenis_04_13-04_09.csv")

datos[datos['dia'] == '9/4/2025']

,tipo_torneo,nombre_torneo,ciudad_torneo,pais_torneo,especificacion_torneo,superficie_torneo,estado_partido,jugadorA,jugadorB,jugadorA2,...,duracion_sets,setsA,setsB,ranking_jugadorA,ranking_jugadorB,genero,dia,hora,cancelado,ganador
28,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,D. Altmaier,R. Gasquet,-,...,Final - 2h 36m,2.0,1,ATP 66.,ATP 166.,masculino,9/4/2025,11:00,no,0
29,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,J. Lehecka,L. Musetti,-,...,Final - 2h 19m,1.0,2,ATP 34.,ATP 7.,masculino,9/4/2025,11:00,no,1
30,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,N. Borges,P. Martinez,-,...,Final - 3h 8m,2.0,1,ATP 41.,ATP 48.,masculino,9/4/2025,11:00,no,0
31,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,T. Etcheverry,A. Davidovich Fokina,-,...,Final - 1h 53m,0.0,2,ATP 53.,ATP 29.,masculino,9/4/2025,11:00,no,1
32,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,F. Cobolli,A. Fils,-,...,Final - 1h 10m,0.0,2,ATP 26.,ATP 14.,masculino,9/4/2025,13:10,no,1
33,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,A. Rublev,G. Monfils,-,...,Final - 1h 43m,2.0,0,ATP 15.,ATP 42.,masculino,9/4/2025,13:35,no,0
34,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,F. Cerundolo,C. Alcaraz,-,...,Final - 1h 38m,1.0,2,ATP 18.,ATP 2.,masculino,9/4/2025,13:55,no,1
35,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,T. Machac,A. de Minaur,-,...,Final - 1h 48m,1.0,2,ATP 22.,ATP 9.,masculino,9/4/2025,14:40,no,1
36,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,R. Bautista Agut,C. Ruud,-,...,Final - 1h 8m,0.0,2,ATP 49.,ATP 8.,masculino,9/4/2025,15:40,no,1
37,ATP 1000,Monte Carlo,Monte Carlo,Monaco,Singles,Tierra batida,Final,N. Djokovic,A. Tabilo,-,...,Final - 1h 27m,0.0,2,ATP 6.,ATP 61.,masculino,9/4/2025,15:55,no,1


Pruebas

In [37]:
for i in datos.columns:
    print(datos[i].value_counts())

tipo_torneo
ATP 500    59
WTA 500    31
Name: count, dtype: int64
nombre_torneo
Stuttgart    31
Munich       30
Barcelona    29
Name: count, dtype: int64
ciudad_torneo
Stuttgart    31
Munich       30
Barcelona    29
Name: count, dtype: int64
pais_torneo
Germany    61
Spain      29
Name: count, dtype: int64
especificacion_torneo
Singles    86
Doubles     4
Name: count, dtype: int64
superficie_torneo
Tierra batida      59
Red clay indoor    31
Name: count, dtype: int64
estado_partido
Final    90
Name: count, dtype: int64
jugadorA
Z. Bergs          3
M. Frech          3
M. Uchijima       3
C. Alcaraz        2
E. Svitolina      2
                 ..
V. Kudermetova    1
L. Bronzetti      1
A. Kalinina       1
E. Jacquemot      1
M. Kessler        1
Name: count, Length: 72, dtype: int64
jugadorB
B. Shelton                 3
D. Altmaier                3
A. de Minaur               2
F. Marozsan                2
S. Korda                   2
                          ..
C. Tauson                